In [3]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
segmentation_transform = A.Compose([
    # Randomly crops the image and resizes it back to the original size
    A.RandomResizedCrop(height=512, width=512, scale=(0.8, 1.0), p=0.5),

    # Random horizontal flipping
    A.HorizontalFlip(p=0.5),

    # Random vertical flipping
    A.VerticalFlip(p=0.5),

    # Randomly changes the brightness, contrast, and saturation of an image
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.02, p=0.5),

    # Randomly rotates the image
    A.Rotate(limit=30, p=0.5, border_mode=cv2.BORDER_CONSTANT),

    # Applies elastic transformations
    A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=0.5),

    # Normalizes the image
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),

    # Converts the image to a PyTorch tensor
    ToTensorV2(),
])

In [4]:
!pip install segmentation-models-pytorch

  Obtaining dependency information for segmentation-models-pytorch from https://files.pythonhosted.org/packages/cb/70/4aac1b240b399b108ce58029ae54bc14497e1bbc275dfab8fd3c84c1e35d/segmentation_models_pytorch-0.3.3-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for timm==0.9.2 from https://files.pythonhosted.org/packages/29/90/94f5deb8d76e24a89813aef95e8809ca8fd7414490428480eda19b133d4a/timm-0.9.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 kB 3.8 MB/s eta 0:00:00
  Obtaining dependency information for munch from https://files.pythonhosted.org/packages/56/b3/7c69b37f03260a061883bec0e7b05be7117c1b1c85f5212c72c8c2bc3c8c/munch-4.0.0-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
import numpy as np
import pandas as pd
import cv2
import os
import segmentation_models_pytorch as smp
import torch
import torch.nn as nn
import torch.optim as optim

***Change the path of best_model.pth***

In [10]:
checkpoint = torch.load('/kaggle/input/model-file/best_model.pth')

In [14]:
import os
os.mkdir('/kaggle/working/predicted_mask')

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
model = smp.Unet(
    encoder_name="resnet34",        
    encoder_weights="imagenet",     
    in_channels=3,                  
    classes=3     
)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

model.load_state_dict(checkpoint['model'])
optimizer.load_state_dict(checkpoint['optimizer'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model = model.to(device)
model.eval()
for i in os.listdir("/kaggle/input/bkai-igh-neopolyp/test/test"):
    img_path = os.path.join("/kaggle/input/bkai-igh-neopolyp/test/test", i)
    ori_img = cv2.imread(img_path)
    ori_img = cv2.cvtColor(ori_img, cv2.COLOR_BGR2RGB)
    ori_w = ori_img.shape[0]
    ori_h = ori_img.shape[1]
    img = cv2.resize(ori_img, (512, 512))
    transformed = segmentation_transform(image=img)
    input_img = transformed["image"]
    input_img = input_img.unsqueeze(0).to(device)
    with torch.no_grad():
        output_mask = model.forward(input_img).squeeze(0).cpu().numpy().transpose(1,2,0)
    mask = cv2.resize(output_mask, (ori_h, ori_w))
    mask = np.argmax(mask, axis=2)
#     new_rgb_mask = np.zeros((*mask.shape, 3)).astype(np.uint8)
#     mask_rgb = mask_to_rgb(mask, color_dict)
#     cv2.imwrite("/kaggle/working/predicted_mask/{}".format(i), mask_rgb)
    cv2.imwrite("/kaggle/working/predicted_mask/{}".format(i), mask)

def rle_to_string(runs):
    return ' '.join(str(x) for x in runs)

def rle_encode_one_mask(mask):
    pixels = mask.flatten()
    pixels[pixels > 225] = 255
    pixels[pixels <= 225] = 0
    use_padding = False
    if pixels[0] or pixels[-1]:
        use_padding = True
        pixel_padded = np.zeros([len(pixels) + 2], dtype=pixels.dtype)
        pixel_padded[1:-1] = pixels
        pixels = pixel_padded
    
    rle = np.where(pixels[1:] != pixels[:-1])[0] + 2
    if use_padding:
        rle = rle - 1
    rle[1::2] = rle[1::2] - rle[:-1:2]
    return rle_to_string(rle)

def rle2mask(mask_rle, shape=(3,3)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (width,height) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

def mask2string(dir):
    ## mask --> string
    strings = []
    ids = []
    ws, hs = [[] for i in range(2)]
    for image_id in os.listdir(dir):
        id = image_id.split('.')[0]
        path = os.path.join(dir, image_id)
        print(path)
        img = cv2.imread(path)[:,:,::-1]
        h, w = img.shape[0], img.shape[1]
        for channel in range(2):
            ws.append(w)
            hs.append(h)
            ids.append(f'{id}_{channel}')
            string = rle_encode_one_mask(img[:,:,channel])
            strings.append(string)
    r = {
        'ids': ids,
        'strings': strings,
    }
    return r


MASK_DIR_PATH = '/kaggle/working/predicted_mask' # change this to the path to your output mask folder
dir = MASK_DIR_PATH
res = mask2string(dir)
df = pd.DataFrame(columns=['Id', 'Expected'])
df['Id'] = res['ids']
df['Expected'] = res['strings']

df.to_csv(r'output.csv', index=False)

/kaggle/working/predicted_mask/3bbc04a3afe1f119f21b248d152b672a.jpeg
/kaggle/working/predicted_mask/7cdf3f33c3ca4d5060a633a8d5b2b2b5.jpeg
/kaggle/working/predicted_mask/1002ec4a1fe748f3085f1ce88cbdf366.jpeg
/kaggle/working/predicted_mask/e73749a0d21db70dd094a7f32574d6c7.jpeg
/kaggle/working/predicted_mask/5e8f14e1e0ae936de314f2d95e6c487f.jpeg
/kaggle/working/predicted_mask/2a365b5574868eb60861ee1ff0b8a4f6.jpeg
/kaggle/working/predicted_mask/1b62f15ec83b97bb11e8e0c4416c1931.jpeg
/kaggle/working/predicted_mask/a6d9ba9d45c3dbc695325ded465efde9.jpeg
/kaggle/working/predicted_mask/a9d45c3dbc695325ded465efde988dfb.jpeg
/kaggle/working/predicted_mask/559c7e610b1531871f2fd85a04faeeb2.jpeg
/kaggle/working/predicted_mask/71f2fd85a04faeeb2b535797395305af.jpeg
/kaggle/working/predicted_mask/39dda50f954ba59c7de13a35276a4764.jpeg
/kaggle/working/predicted_mask/a15fc656702fa602bb3c7abacdbd7e6a.jpeg
/kaggle/working/predicted_mask/50534bca540e24f489284b8e6953ad88.jpeg
/kaggle/working/predicted_mask/5c1